### L’algorithme utiliser  pour le clustring est le k-mean

# Question 2.1 

In [1]:
import pandas as pd 
rawDfPh3 = pd.read_csv("files/Phase3.csv")

### Description  des donner  

In [2]:
rawDfPh3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22544 entries, 0 to 22543
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     22544 non-null  int64  
 1   protocol_type                22544 non-null  object 
 2   service                      22544 non-null  object 
 3   flag                         22544 non-null  object 
 4   src_bytes                    22544 non-null  int64  
 5   dst_bytes                    22544 non-null  int64  
 6   land                         22544 non-null  int64  
 7   wrong_fragment               22544 non-null  int64  
 8   urgent                       22544 non-null  int64  
 9   hot                          22544 non-null  int64  
 10  num_failed_logins            22544 non-null  int64  
 11  logged_in                    22544 non-null  int64  
 12  num_compromised              22544 non-null  int64  
 13  root_shell      

In [3]:
print(rawDfPh3.describe())

           duration     src_bytes     dst_bytes          land  wrong_fragment  \
count  22544.000000  2.254400e+04  2.254400e+04  22544.000000    22544.000000   
mean     218.859076  1.039545e+04  2.056019e+03      0.000311        0.008428   
std     1407.176612  4.727864e+05  2.121930e+04      0.017619        0.142599   
min        0.000000  0.000000e+00  0.000000e+00      0.000000        0.000000   
25%        0.000000  0.000000e+00  0.000000e+00      0.000000        0.000000   
50%        0.000000  5.400000e+01  4.600000e+01      0.000000        0.000000   
75%        0.000000  2.870000e+02  6.010000e+02      0.000000        0.000000   
max    57715.000000  6.282565e+07  1.345927e+06      1.000000        3.000000   

             urgent           hot  num_failed_logins     logged_in  \
count  22544.000000  22544.000000       22544.000000  22544.000000   
mean       0.000710      0.105394           0.021647      0.442202   
std        0.036473      0.928428           0.150328      0.

### Gestion des valeur redondante 

In [4]:
rawDfPh3=  rawDfPh3.drop_duplicates(keep='first').reset_index(drop=True)# dellting of all the rows that are duplicate

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22487 entries, 0 to 22486
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     22487 non-null  int64  
 1   protocol_type                22487 non-null  object 
 2   service                      22487 non-null  object 
 3   flag                         22487 non-null  object 
 4   src_bytes                    22487 non-null  int64  
 5   dst_bytes                    22487 non-null  int64  
 6   land                         22487 non-null  int64  
 7   wrong_fragment               22487 non-null  int64  
 8   urgent                       22487 non-null  int64  
 9   hot                          22487 non-null  int64  
 10  num_failed_logins            22487 non-null  int64  
 11  logged_in                    22487 non-null  int64  
 12  num_compromised              22487 non-null  int64  
 13  root_shell      

### Gestion des valeur null

In [5]:
print(' affichage des valeur null par column : ', rawDfPh3.isnull().sum()) #afficher lles valeur null par column 
transformedDf= rawDfPh3.fillna(rawDfPh3.median())# remplire les valeur nulle par la valeur median , nous navont pas de valuer null pour notre cas 

 affichage des valeur null par column :  duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
di

### Transformation des donner 
utilisation dune codification one hoting 

In [6]:
txtDataColumn = ['protocol_type','service','flag']
transformedDf = pd.get_dummies(transformedDf,columns=txtDataColumn) 

## aplilcation du k mean 

In [7]:
from sklearn.cluster import KMeans
modelKmean=KMeans(n_clusters=2,  init='k-means++', random_state=10 )
modelKmean.fit(transformedDf)

KMeans(n_clusters=2, random_state=10)

## Question 2 
### on utilisera le coefficient de sihlouette 

In [8]:

from sklearn.metrics import silhouette_score
shilouette =silhouette_score(transformedDf,modelKmean.labels_,  metric = 'euclidean')
print("la shlouette de notre k-mean : ", round(shilouette,3))

la shlouette de notre k-mean :  1.0


### enregistrement du fichier fromPhase3.csv qui est le fichier phase2.cvs avec une column class obtenue de apartire du clustring 

In [9]:
from collections import Counter
labels = modelKmean.labels_
print( Counter(labels))
rawDfPh3['class'] = labels #ajout dune colone class au donner de la phase 3
txtDataColumn = ['protocol_type','service','flag']
rawDfPh3 = rawDfPh3.drop(txtDataColumn, axis= 1)
rawDfPh3.to_csv('files/fromPhase3.csv',index = True, index_label="index")
print('le ficheir a bien etait enregister ')

Counter({0: 22485, 1: 2})
le ficheir a bien etait enregister 


## Question 3 
### Oui, pour avoir un modèle plus robuste en doit faire un modèle qui ces entrainer sur une quantité de donner importante,  pour réaliser cela on doit entrainer un nouveaux modèle sur les donner du fichier phase2 ainsi que ceux de pahse3, en utilisera le fichier créer dans la phase 3 ("fromPhase3.csv") pour l’implémentation de la solution, la solution sera implémenter dans un autre fichier nommer "implementation.ipynb"

## J’ai préfère faire un nouveau fichier .ipynb aux lieux de de tout faire dans un même fichier Jupiter pour des soucis de clarté, d'où l’exigence d’UN fichier supplémentaire ‘fromPhase3.csv ‘ pour stocker le résultat de cette phase qui seront utiliser dans ‘implementation.ipynb’